In [ ]:
%pip install -Uqq sagemaker

In [ ]:
%pip install modin

In [ ]:
import boto3
import json
import sagemaker
from pathlib import Path
from sagemaker.pytorch import PyTorch  # PyTorch Estimator for running our training job

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
bucket = sess.default_bucket()  # default bucket name
account_id = sess.account_id()

In [ ]:
train_data_path = "s3://nvidia-aws-fraud-detection-demo-1/output121_clean/train/"
test_data_path = "s3://nvidia-aws-fraud-detection-demo-1/output121_clean/test-small/"

In [ ]:
USE_GPU = True

gpu_job = PyTorch(
    source_dir="src",
    entry_point="train.py",
    framework_version="2.3",
    py_version="py311",
    role=role,
    environment={"USE_GPU": str(USE_GPU)},
    hyperparameters={                                
        "boost_round": 100,
        "train_data_path": train_data_path,
        "test_data_path": test_data_path,
    },
    instance_type="ml.g5.12xlarge",
    instance_count = 1,            
    max_run=1000,                                    
    keep_alive_period_in_seconds=300                 
)

In [ ]:
gpu_job.fit()

In [ ]:
USE_GPU = False

cpu_job = PyTorch(
    source_dir="src",
    entry_point="train.py",
    framework_version="2.3",
    py_version="py311",
    role=role,
    environment={"USE_GPU": str(USE_GPU)},
    hyperparameters={                                
        "boost_round": 100,
        "train_data_path": train_data_path,
        "test_data_path": test_data_path,
    },
    instance_type="ml.r5.12xlarge",
    instance_count = 2,            
    max_run=1000,                                    
    keep_alive_period_in_seconds=300                 
)

In [ ]:
cpu_job.fit()